In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Recommend.csv',names=['user_id', 'movie_id', 'rating', 'timestamp'])
df

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [3]:
from sklearn.model_selection import train_test_split
n_users = df.user_id.unique().shape[0] 
n_movies = df.movie_id.unique().shape[0]
train_data, test_data = train_test_split(df, test_size=0.25)

In [8]:
train_data_matrix = np.zeros((n_users, n_movies))
for line in train_data.itertuples():
    #[user_id index, movie_id index] = given rating.
    train_data_matrix[line[1]-1, line[2]-1] = line[3] 
train_data_matrix

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [9]:
test_data_matrix = np.zeros((n_users, n_movies))
for line in test_data.itertuples():
    #[user_id index, movie_id index] = given rating.
    test_data_matrix[line[1]-1, line[2]-1] = line[3]
test_data_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
from sklearn.metrics import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
movie_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')
print(user_similarity)
print(movie_similarity)
mean_user_rating = train_data_matrix.mean(axis=1)[:, np.newaxis]
print(mean_user_rating)
ratings_diff = (train_data_matrix - mean_user_rating) 
print(ratings_diff)
user_pred = mean_user_rating + user_similarity.dot(ratings_diff) / np.array([np.abs(user_similarity).sum(axis=1)]).T
print(user_pred)

[[0.         0.86246612 0.98529379 ... 0.8819337  0.87741315 0.72522576]
 [0.86246612 0.         0.85924059 ... 0.87175558 0.85487476 0.91242967]
 [0.98529379 0.85924059 0.         ... 0.91162794 0.87897186 0.97783285]
 ...
 [0.8819337  0.87175558 0.91162794 ... 0.         0.9669666  0.88321032]
 [0.87741315 0.85487476 0.87897186 ... 0.9669666  0.         0.87964494]
 [0.72522576 0.91242967 0.97783285 ... 0.88321032 0.87964494 0.        ]]
[[0.         0.66367268 0.69440049 ... 1.         0.94805195 0.94805195]
 [0.66367268 0.         0.80060952 ... 1.         0.90913262 1.        ]
 [0.69440049 0.80060952 0.         ... 1.         1.         0.89089105]
 ...
 [1.         1.         1.         ... 0.         1.         1.        ]
 [0.94805195 0.90913262 1.         ... 1.         0.         1.        ]
 [0.94805195 1.         0.89089105 ... 1.         1.         0.        ]]
[[0.43638526]
 [0.11058264]
 [0.05648038]
 [0.04875149]
 [0.22413793]
 [0.35850178]
 [0.7491082 ]
 [0.1117717 ]
